## 1. Libraries

Python version: 3.8.20

In [1]:
import pandas as pd
import numpy as np
import os
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [2]:
pd.set_option('display.max_rows', 1000); pd.set_option('display.max_columns', 1000); pd.set_option('display.width', 1000)

## 2. Functions

In [3]:
# ML models except XGBoost
def train_model(model, x_train, y_train, x_test, y_test):
    # Fit model
    model.fit(x_train, y_train)
    # Generate predictions
    y_pred = model.predict(x_test)
    # MSE Score
    mse = mean_squared_error(y_test, y_pred)
    # RMSE Score
    rmse = np.sqrt(mse)
    # R^2 Score
    r2 = r2_score(y_test, y_pred)
    
    print(f'Model = {model}, Root Mean Squared Error (RMSE): {rmse}, R² Score = {r2}')

    return model

In [4]:
# XGboost
def train_model_xgb(xgb, x_train, y_train, x_test, y_test):
    # Fit model
    xgb.fit(x_train, y_train)
    # Generate predictions
    y_pred = xgb.predict(x_test)
    # MSE Score
    mse = mean_squared_error(y_test, y_pred)
    # RMSE Score
    rmse = np.sqrt(mse)
    # R^2 Score
    r2 = r2_score(y_test, y_pred)
    
    print(f'Model = {xgb}, Root Mean Squared Error (RMSE): {rmse}, R² Score = {r2}')

    return xgb

## 3. Datasets

In [5]:
# Load approval date datasets
tmp_path = "./Kaggle_HDB/" 
approval_files = [
    tmp_path + str(x) for x in os.listdir(tmp_path) if \
        x.startswith("resale-flat-prices-based-on-approval-date-")] 

tmp_approval_data = [pd.read_csv(tmp_file) for tmp_file in approval_files ]
tmp_approval_data = pd.concat(tmp_approval_data)

In [6]:
tmp_approval_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 656851 entries, 0 to 369650
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                656851 non-null  object 
 1   town                 656851 non-null  object 
 2   flat_type            656851 non-null  object 
 3   block                656851 non-null  object 
 4   street_name          656851 non-null  object 
 5   storey_range         656851 non-null  object 
 6   floor_area_sqm       656851 non-null  float64
 7   flat_model           656851 non-null  object 
 8   lease_commence_date  656851 non-null  int64  
 9   resale_price         656851 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 55.1+ MB


In [7]:
tmp_approval_data.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0


In [8]:
# Load registration date datasets
tmp_path = "./Kaggle_HDB/" 
registration_files = [
    tmp_path + str(x) for x in os.listdir(tmp_path) if \
        x.startswith("resale-flat-prices-based-on-registration-date-")] 

tmp_registration_data = [pd.read_csv(tmp_file) for tmp_file in registration_files ]
tmp_registration_data = pd.concat(tmp_registration_data)

In [9]:
tmp_registration_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 169730 entries, 0 to 52202
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                169730 non-null  object 
 1   town                 169730 non-null  object 
 2   flat_type            169730 non-null  object 
 3   block                169730 non-null  object 
 4   street_name          169730 non-null  object 
 5   storey_range         169730 non-null  object 
 6   floor_area_sqm       169730 non-null  float64
 7   flat_model           169730 non-null  object 
 8   lease_commence_date  169730 non-null  int64  
 9   remaining_lease      117527 non-null  object 
 10  resale_price         169730 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 15.5+ MB


In [10]:
tmp_registration_data.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0


## 4. Data Processing

Assumption 1: The file labelled 'approval date' follows the old naming convention while the current file is named 'registration date’.

Assumption 2: All categorical columns in the datasets contain the correct categories except for issues related to formatting inconsistencies.

### (i) Find the total lease (years) for each flat in registration data dataset

In [11]:
# Drop rows with na values and duplicates, if any
registration_data = tmp_registration_data.copy(deep=True)
registration_data = registration_data.dropna() 
registration_data = registration_data.drop_duplicates() 

In [12]:
tmp_registration_data.shape[0],registration_data.shape[0] #30.89% of the dataset has been removed

(169730, 117293)

In [13]:
# Convert month to datetime format and extract the month and year into new columns
registration_data['month'] = pd.to_datetime(registration_data['month'])
registration_data['mm'] =  registration_data['month'].dt.month
registration_data['yy'] =  registration_data['month'].dt.year

In [14]:
#  Find the period (years) from lease commencement to stated month
registration_data['years_since_commence'] = registration_data['yy'] - registration_data['lease_commence_date']

# If remaining_lease col contains only numeric values, assume it as years and assign to remaining_lease_years col
# Else, contains words like 'years', extract the value and assign to remaining_lease_years col
registration_data['remaining_lease_years'] = pd.to_numeric(registration_data['remaining_lease'], errors='coerce')
registration_data['remaining_lease_years'] = registration_data['remaining_lease_years'].fillna(
    registration_data['remaining_lease'].str.extract('(\d+) years')[0].astype(float))

# If remaining_lease col contains words like 'months', extract the value and assign to remaining_lease_months col
# Else, assgin 0 to remaining_lease_months col
registration_data['remaining_lease_months'] = registration_data['remaining_lease'].str.extract('(\d+) months').astype(float)
registration_data['remaining_lease_months'] = registration_data['remaining_lease_months'].fillna(0)

# Find total lease in years
registration_data['remaining_lease_in_years'] = registration_data['remaining_lease_years'] + (registration_data['remaining_lease_months'] / 12)
registration_data['total_lease_in_years'] = registration_data['years_since_commence'] + registration_data['remaining_lease_in_years']

In [15]:
registration_data.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,mm,yy,years_since_commence,remaining_lease_years,remaining_lease_months,remaining_lease_in_years,total_lease_in_years
0,2015-01-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,1,2015,29,70.0,0.0,70.0,99.0
1,2015-01-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,1,2015,34,65.0,0.0,65.0,99.0
2,2015-01-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,1,2015,35,64.0,0.0,64.0,99.0
3,2015-01-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,1,2015,36,63.0,0.0,63.0,99.0
4,2015-01-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,1,2015,35,64.0,0.0,64.0,99.0


In [16]:
# Get the total lease (years) for each flat
total_lease = registration_data.groupby(['block', 'street_name','lease_commence_date'])['total_lease_in_years'].first()
total_lease = total_lease.reset_index()

In [17]:
total_lease.head()

,block,street_name,lease_commence_date,total_lease_in_years
0,1,BEACH RD,1979,99.0
1,1,BEDOK STH AVE 1,1976,98.0
2,1,CHAI CHEE RD,1983,99.0
3,1,CHANGI VILLAGE RD,1980,99.0
4,1,DELTA AVE,1983,99.0


### (ii) Find the total lease (years) for each flat in approval data dataset

In [18]:
# Drop rows with na values and duplicates, if any
approval_data = tmp_approval_data.copy(deep=True)
approval_data = approval_data.dropna() 
approval_data = approval_data.drop_duplicates() 

In [19]:
tmp_approval_data.shape[0],approval_data.shape[0] #0.20% of the dataset has been removed

(656851, 655512)

In [20]:
# Convert month to datetime format and extract the month and year into new columns
approval_data['month'] = pd.to_datetime(approval_data['month'])
approval_data['mm'] =  approval_data['month'].dt.month
approval_data['yy'] =  approval_data['month'].dt.year

In [21]:
# Get total lease for each flat
approval_data_merge  = approval_data.merge(total_lease, how='left', left_on=['block', 'street_name', 'lease_commence_date'],
                                     right_on=['block', 'street_name', 'lease_commence_date'])

In [22]:
# Drop na value for total_lease_in_years col 
approval_data = approval_data_merge.dropna().reset_index(drop=True)

In [23]:
approval_data_merge.shape[0], approval_data.shape[0] #3.06% of the datset has been removed.

(655512, 635478)

### (iii) Find the remaining lease (years) for each flat in approval data dataset

In [24]:
# Get remaining lease for each flat
approval_data['remaining_lease'] = approval_data['total_lease_in_years'] - approval_data['yy'] + approval_data['lease_commence_date']

In [25]:
approval_data.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,mm,yy,total_lease_in_years,remaining_lease
0,1990-01-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,1,1990,98.0,84.0
1,1990-01-01,ANG MO KIO,3 ROOM,211,ANG MO KIO AVE 3,01 TO 03,67.0,NEW GENERATION,1977,46000.0,1,1990,99.0,86.0
2,1990-01-01,ANG MO KIO,3 ROOM,202,ANG MO KIO AVE 3,07 TO 09,67.0,NEW GENERATION,1977,42000.0,1,1990,99.0,86.0
3,1990-01-01,ANG MO KIO,3 ROOM,235,ANG MO KIO AVE 3,10 TO 12,67.0,NEW GENERATION,1977,38000.0,1,1990,99.0,86.0
4,1990-01-01,ANG MO KIO,3 ROOM,235,ANG MO KIO AVE 3,04 TO 06,67.0,NEW GENERATION,1977,40000.0,1,1990,99.0,86.0


### (iv) Combine registration date and approval date datsets and select factors that determine the prices of HDB

In [26]:
# Identify factors that determine the prices of HDB
registration_data = registration_data[['mm', 'yy', 'town', 'flat_type', 'block', 'street_name', 
                                       'storey_range', 'floor_area_sqm', 'flat_model','remaining_lease_years', 'resale_price']]

approval_data = approval_data[['mm', 'yy', 'town', 'flat_type', 'block', 'street_name', 
                                       'storey_range', 'floor_area_sqm', 'flat_model','remaining_lease', 'resale_price']]

# Rename remaining_lease_years col back to remaining_lease to ensure same col name as approval_data
registration_data.rename(columns={"remaining_lease_years":"remaining_lease"}, inplace=True)

In [27]:
# Join registration date and approval date datasets
hdb_data = pd.concat([registration_data, approval_data])

In [28]:
# Standardlise the catergory value for flat_model and flat_type and rename mm as month and yy as year
hdb_data['flat_model'] = hdb_data['flat_model'].str.upper()
hdb_data['flat_type'] = hdb_data['flat_type'].str.replace('MULTI-GENERATION', 'MULTI GENERATION')
hdb_data.rename(columns={'mm': 'month', 'yy': 'year'}, inplace=True)

In [29]:
hdb_data.shape[0]

752771

In [30]:
hdb_data.head()

,month,year,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,remaining_lease,resale_price
0,1,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,IMPROVED,70.0,255000.0
1,1,2015,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,NEW GENERATION,65.0,275000.0
2,1,2015,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,NEW GENERATION,64.0,285000.0
3,1,2015,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,NEW GENERATION,63.0,290000.0
4,1,2015,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,NEW GENERATION,64.0,290000.0


## 5. Machine Learning

### (i) Train, Test Dataset

In [31]:
train_data = hdb_data.drop(columns='resale_price')
test_data = hdb_data['resale_price']

One-hot encoding is generally preferred for encoding categorical variables, especially when the number of categories is small (typically 5 or fewer). However, in this case, all categorical columns have more than 5 categories, which makes one-hot encoding impractical due to the high dimensionality it would introduce. For example, the town column has 26 categories and creating 26 additional features would significantly increase the number of dimensions.

Hence, I used label encoding. However, it is important to note that this method may not provide as accurate predictions as one-hot encoding. Label encoding assigns numeric values to categories but these values do not have an inherent ordinal relationship. For instance, encoding "Jan" as 1 and "Feb" as 2 could incorrectly imply that "Feb" is better than "Jan." The model might interpret this numeric order as meaningful, even though there is no ordinal hierarchy between the months.

In [32]:
# Label encoding for town, flat_type, block, street_name, storey_range and flat_model
le_1 = LabelEncoder()
train_data['town'] = le_1.fit_transform(train_data['town'])
le_2 = LabelEncoder()
train_data['flat_type'] = le_2.fit_transform(train_data['flat_type'])
le_3 = LabelEncoder()
train_data['block'] = le_3.fit_transform(train_data['block'])
le_4 = LabelEncoder()
train_data['street_name'] = le_4.fit_transform(train_data['street_name'])
le_5 = LabelEncoder()
train_data['storey_range'] = le_5.fit_transform(train_data['storey_range'])
le_6 = LabelEncoder()
train_data['flat_model'] = le_6.fit_transform(train_data['flat_model'])

In [33]:
train_data.head()

,month,year,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,remaining_lease
0,1,2015,0,2,237,15,2,60.0,4,70.0
1,1,2015,0,2,1491,12,0,68.0,11,65.0
2,1,2015,0,2,194,15,0,69.0,11,64.0
3,1,2015,0,2,1173,12,0,68.0,11,63.0
4,1,2015,0,2,1521,12,2,68.0,11,64.0


In [34]:
# Split into 80% train and 20% test
x_train, x_test, y_train, y_test = train_test_split(train_data, test_data, test_size=0.2, random_state=42)

In [35]:
x_train.sample(3)

,month,year,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,remaining_lease
70586,6,2020,0,2,94,14,3,67.0,11,57.0
165322,11,1997,21,3,402,387,4,93.0,11,87.0
432466,11,2004,13,3,1822,256,2,86.0,9,94.0


In [36]:
x_test.sample(3)

,month,year,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,remaining_lease
120423,6,1996,22,4,159,398,0,122.0,4,91.0
261283,9,1999,8,4,1096,113,1,122.0,4,92.0
15493,11,2015,11,5,436,203,1,148.0,6,75.0


### (ii) Machine Learning Models

In [37]:
# Random Forest
rf = RandomForestRegressor(random_state=42)
train_model(rf, x_train, y_train, x_test, y_test)

Model = RandomForestRegressor(random_state=42), Root Mean Squared Error (RMSE): 19735.13789939973, R² Score = 0.9807871175430473


RandomForestRegressor(random_state=42)

In [38]:
# XGBoost
xgb = XGBRegressor(random_state=42)
train_model_xgb(xgb, x_train, y_train, x_test, y_test)

Model = XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...), Root Mean Squared Error (RMSE): 22955.735302753044, R² Score = 0.9740047132916885


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [39]:
# Linear Regression
lr = LinearRegression()
train_model(lr, x_train, y_train, x_test, y_test)

Model = LinearRegression(), Root Mean Squared Error (RMSE): 74111.18199913905, R² Score = 0.7290559147158233


LinearRegression()

Interpretation: 

R²: Indicates how well the model explains the variance in Resale Price, R² closer to 1 is better as it means the model is explaining most of the the variability in Resale Price.

RMSE: Measures the average error between the actual and predicted values, lower RMSE is better as it means the predicted values are closer to the actual values.

Best Prediction Model: Random Forest with RMSE: 19735.14 (lowest among all) and R² Score = 0.98 (highest among all)

## 6. Results

In [40]:
# Get predictions
pred = rf.predict(x_test)
pred_df = pd.DataFrame(pred, columns=['PREDICTION'])
x_test = x_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
results = pd.concat([x_test,y_test, pred_df], axis=1)
results['town'] = le_1.inverse_transform(results['town'] )
results['flat_type'] = le_2.inverse_transform(results['flat_type'] )
results['block'] = le_3.inverse_transform(results['block'] )
results['street_name'] = le_4.inverse_transform(results['street_name'] )
results['storey_range'] = le_5.inverse_transform(results['storey_range'] )
results['flat_model'] = le_6.inverse_transform(results['flat_model'] )

In [41]:
results.sample(5)

,month,year,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,remaining_lease,resale_price,PREDICTION
51235,2,1995,ANG MO KIO,3 ROOM,419,ANG MO KIO AVE 10,04 TO 06,74.0,NEW GENERATION,83.0,103000.0,98620.0
124884,9,2007,BUKIT MERAH,4 ROOM,104A,DEPOT RD,07 TO 09,100.0,MODEL A,92.0,418000.0,379156.0
16689,11,1995,TAMPINES,EXECUTIVE,944,TAMPINES AVE 5,01 TO 03,146.0,MAISONETTE,92.0,490000.0,492270.0
131519,10,1997,BUKIT BATOK,3 ROOM,233,BT BATOK EAST AVE 5,04 TO 06,67.0,NEW GENERATION,87.0,180000.0,164255.0
133628,10,2018,BUKIT PANJANG,2 ROOM,488,SEGAR RD,13 TO 15,47.0,MODEL A,95.0,223000.0,237610.0


In [42]:
# Save predictions
results.to_csv('./HDB Price Predictions.csv')

In [43]:
# Save model
pickle.dump(rf, open('Random Forest Predictive Model.pkl', 'wb'))

## 7. Insights

Feature importance from the trained Random Forest model

In [44]:
# Visualize the feature importance
feature_importance = rf.feature_importances_ 
feature_importance_df = pd.DataFrame({ 'Feature': x_train.columns, 
                                      'Importance': feature_importance }).sort_values(by='Importance', ascending=False)
feature_importance_df

,Feature,Importance
1,year,0.378499
7,floor_area_sqm,0.354946
3,flat_type,0.103340
2,town,0.059535
6,storey_range,0.025852
5,street_name,0.023096
9,remaining_lease,0.020319
4,block,0.015257
8,flat_model,0.011243
0,month,0.007913


Top 3 most important factors contributing to resale price: 
1) year (37.85%)
2) floor area sqm (35.49%)
3) flat type (10.33%)

In [45]:
# Correlation between resale price and year
correlation_year = hdb_data[['resale_price', 'year']].corr()
print(correlation_year)
# 0.58, moderate positive correlation for resale price as year increases.

              resale_price      year
resale_price      1.000000  0.575808
year              0.575808  1.000000


In [46]:
# Correlation between resale price and floor area sqm
correlation_floor_area_sqm = hdb_data[['resale_price', 'floor_area_sqm']].corr()
print(correlation_floor_area_sqm)
# 0.64, moderate positive correlation for resale price  as floor area sqm increases.

                resale_price  floor_area_sqm
resale_price        1.000000        0.635736
floor_area_sqm      0.635736        1.000000


In [47]:
# Correlation between resale price and flat type
le_flat_type = LabelEncoder()
flat_type_to_resale_price = hdb_data[['resale_price', 'flat_type']].reset_index(drop=True)
flat_type_to_resale_price['flat_type'] = le_flat_type.fit_transform(flat_type_to_resale_price['flat_type'])
correlation_flat_type = flat_type_to_resale_price[['resale_price', 'flat_type']].corr()
print(le_flat_type.classes_)
print(correlation_flat_type) 
# 0.65, moderate positive correlation for resale price as flat type increases from 1 ROOM to MULTI GENERATION.

['1 ROOM' '2 ROOM' '3 ROOM' '4 ROOM' '5 ROOM' 'EXECUTIVE'
 'MULTI GENERATION']
              resale_price  flat_type
resale_price      1.000000   0.653212
flat_type         0.653212   1.000000
